In [1]:

def Data_check(Data, headers=False):
    headers = Data.columns if type(headers) is bool else headers 
    bool_array = [pd.notna(Data[col_name]) for col_name in headers]
    check = np.flipud(np.rot90(bool_array))
    return check

def Find_least_error(Data, Checked_data, headers, error_headers, pm_seperation=True):
    least_error_array = []
    for row, boolean in enumerate(Checked_data):
        previous_error = np.inf
        for dataset, error_header in enumerate(error_headers):
            if sum(boolean) == 0:
                lowest_error = False
                break
            if boolean[dataset]:
                if pm_seperation:
                    current_error = Data[error_header[0]][row] + Data[error_header[1]][row]
                else:
                    current_error = 2*Data[error_header][row]
                if current_error < previous_error:
                    previous_error = current_error
                    lowest_error = headers[dataset]
        least_error_array.append(lowest_error)
    return least_error_array

def Ensure_all(All_variables):
    total_info = []
    num_stars = len(All_variables[0])
    for star_index in range(num_stars):
        object_info = [star_index]
        eliminate = False
        for variable_source in All_variables:
            eliminate = type(variable_source[star_index]) is bool
            if eliminate:
                break
            object_info.append(variable_source[star_index])
        if eliminate:
            continue
        total_info.append(object_info)
    return total_info

def Get_filtered_arrays(Data, Filtered_headers):
    Total_data = []
    for row in Filtered_headers:
        row_data = []
        for index in range(1, len(row)):
            row_data.append(Data[row[index]][row[0]])
        Total_data.append(row_data)
    return np.array(Total_data)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.constants as const
import seaborn as sns

In [3]:
%matplotlib notebook

In [4]:
def Create_dataframes(filenames, directory):
    dataframes = []
    for name in filenames:
        file_directory = f'{directory}/{name}.csv'
        data = pd.read_csv(file_directory)
        dataframes.append(data)
    return dataframes

Main_directory = 'C:/Users/Iwan/Documents/GitHub/Iwan-and-George-Year-4-Project/Exoplanets'
All_stars_directory = f'{Main_directory}/Sorted Data/Stars'
All_planets_directory = f'{Main_directory}/Sorted Data/Planets'

star_filenames = ['Mega All Stars','Gaia All Stars', 'Exo All Stars', 
             'Fulton All Stars', 'VanEylen All Stars', 'BASTA All Stars']

planet_filenames = ['Mega All Planets', 'VanHo All Planets', 
                    'Fulton All Planets', 'VanEylen All Planets']

starframes = Create_dataframes(star_filenames, All_stars_directory)
planetframes = Create_dataframes(planet_filenames, All_planets_directory)

In [5]:
Teff_headers = ['Teff (Gaia)', 'Teff (Exo)', 'Teff (Fulton)', 'Teff (Van Eylen)']
Rad_headers = ['Radius (Gaia)', 'Radius (Fulton)', 'Radius (Van Eylen)']

Teff_check = Data_check(All_stars_data, Teff_headers)
Rad_check = Data_check(All_stars_data, Rad_headers)

Teff_error_headers = [['Teff pe (Gaia)', 'Teff me (Gaia)'], 
                      ['Teff pe (Exo)', 'Teff me (Exo)'], 
                      ['Teff pe (Fulton)', 'Teff me (Fulton)'],
                      ['Teff pe (Van Eylen)', 'Teff me (Van Eylen)']]

Rad_error_headers = [['Rad pe (Gaia)', 'Rad me (Gaia)'],
                     ['Rad pe (Fulton)', 'Rad me (Fulton)'],
                     ['Rad pe (Van Eylen)', 'Rad me (Van Eylen)']]
            
Best_teff_sources = Find_least_error(All_stars_data, Teff_check, Teff_headers, Teff_error_headers)
Best_rad_sources = Find_least_error(All_stars_data, Rad_check, Rad_headers, Rad_error_headers)

All_info = [Best_teff_sources, Best_rad_sources]
Filtered_headers = Ensure_all(All_info)

NameError: name 'All_stars_data' is not defined

In [ ]:
Array = Get_filtered_arrays(All_stars_data, Filtered_headers)
Teff = Array[:,0]
Rad = Array[:,1]
Luminosity = Rad**2*(Teff/5772)**4

plt.figure()
plt.plot(Teff, Luminosity, 'r.', markersize=1)
plt.plot(5772, 1, 'g.', label='Sol')
plt.ylabel('Luminosity (Solar Luminosities)')
plt.xlabel('Effective Temperature (K)')
plt.xlim(2000, 7500)
plt.ylim(0.00001, 100000)
plt.yscale('log')
plt.gca().invert_xaxis()
plt.legend()
plt.show()

sns.set_theme(style="ticks")

sns.jointplot(x=Teff, y=Luminosity, xlim=(7000, 3000), ylim=(-1, 7.5), kind="hex", color="#4CB391")

In [ ]:
PSR_headers = ['PSR (VanHo)', 'PSR (Fulton)', 'PSR (Van Eylen)']
PSR_check = Data_check(All_planets_data, PSR_headers)

print(All_planets_data['Host Gaia ID'])
Planet_data = []
for P_index, IDp in enumerate(All_planets_data['Host Gaia ID']):
    for S_index, IDs in enumerate(All_stars_data['designation']):
        print([IDp, IDs])
        if IDp == IDs:
            P_radius = All_planets_data['PSR (VanHo)'][P_index]*All_stars_data['Radius (Gaia)'][S_index]
            P_period = All_planets_data['P (VanHo)'][P_index]
            Planet_data.append([P_radius, P_period])
            
print(Planet_data)